In [1]:
# Mount Drive

from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 14 15:30:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    32W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Download dataset

# Relies on HIDE dataset found here: https://github.com/joanshen0508/HA_deblur
!wget --no-check-certificate \
    "https://www.dropbox.com/s/04w3wqxcuin9dy8/HIDE_dataset.zip?dl=0" \
    -O "/tmp/HIDE_dataset.zip"

--2022-11-14 15:30:37--  https://www.dropbox.com/s/04w3wqxcuin9dy8/HIDE_dataset.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/04w3wqxcuin9dy8/HIDE_dataset.zip [following]
--2022-11-14 15:30:39--  https://www.dropbox.com/s/raw/04w3wqxcuin9dy8/HIDE_dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc13da0cc81d93d4c9695ccf24b5.dl.dropboxusercontent.com/cd/0/inline/BwuLaxhcU3IV1tOEq7L6ax-dUdoQFjJ5YixyFbzK_50vmbhhtiDsHD5qf2xyLLcS4Q6t44AwSZ-WLSGS7uR77416OHKjd3VNSrXGiXNHBmhNMMzALQVMz-RSROvUfp0VUrC8dZX2N_15wYZrMwSjU_MGntI6HN6TQDIXZxYwjyw9iw/file# [following]
--2022-11-14 15:30:39--  https://uc13da0cc81d93d4c9695ccf24b5.dl.dropboxusercontent.com/cd/0/inline/BwuLaxhcU3IV1tOEq7L6ax-dUdoQFjJ5YixyFbzK_50vmbhhtiDsHD5qf2xyLLc

In [4]:
# Exctract dataset

import zipfile

with zipfile.ZipFile('/tmp/HIDE_dataset.zip', 'r') as zip_ref:
  zip_ref.extractall('/tmp')

In [5]:
# Imports

import os
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [6]:
# Check GPU

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [7]:
# Prepare paths for dataset

width, height = 256, 144
# width, height = 848, 480

DATASET_DIR = "/tmp/HIDE_dataset/"

TRAIN_FNAME = DATASET_DIR+"train.txt"
# TEST_FNAME  = DATASET_DIR+"test.txt"

GT_DIR    = DATASET_DIR+"GT/"
TRAIN_DIR = DATASET_DIR+"train/"
# TEST_DIR  = DATASET_DIR+"test/test-all/"

def get_filenames(txt_path):
   with open(txt_path, "r") as f:
       filenames = [l.rstrip("\n") for l in f.readlines()]
       return filenames

train_names = get_filenames(TRAIN_FNAME)
# test_names  = get_filenames(TEST_FNAME)

In [8]:
# Create a generator (a Sequence object) because dataset can't fit in GPU memory

from tensorflow.keras.utils import Sequence
import random

BATCH_SIZE = 64

# based on:
# https://stackoverflow.com/questions/62916904/failed-copying-input-tensor-from-cpu-to-gpu-in-order-to-run-gatherve-dst-tensor

class TrainGenerator(Sequence):
    def __init__(self, fnames, batch_size):
        self.fnames = fnames
        self.batch_size = batch_size
        self.rng = random.Random(42)

    def __len__(self):
        return int(np.ceil(len(self.fnames) / self.batch_size))

    def __getitem__(self, idx):
        batch_names = self.fnames[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        batch_x = np.empty((self.batch_size, width, height, 3), dtype=np.float32, order='C')
        batch_y = np.empty((self.batch_size, width, height, 3), dtype=np.float32, order='C')
        
        for i, fname in enumerate(batch_names):
            gt = Image.open(GT_DIR+fname)
            gt = gt.resize((width,height), resample=Image.LANCZOS)#resample=Image.Resampling.LANCZOS)
            gt = np.asarray(gt, dtype=np.float32)/255
            gt = gt.transpose((1, 0, 2))
            batch_y[i,...] = gt
            blur = Image.open(TRAIN_DIR+fname)
            blur = blur.resize((width,height), resample=Image.LANCZOS)#resample=Image.Resampling.LANCZOS)
            blur = np.asarray(blur, dtype=np.float32)/255
            blur = blur.transpose((1, 0, 2))
            batch_x[i,...] = blur
        
        return (batch_x, batch_y)
    
    def on_epoch_end(self):
        self.rng.shuffle(self.fnames)

In [9]:
# Train-test split

from sklearn.model_selection import train_test_split

f_train, f_val = train_test_split(
    train_names, test_size=0.2, random_state=42)

gen_train = TrainGenerator(f_train, BATCH_SIZE)
gen_val   = TrainGenerator(f_val,  BATCH_SIZE)

In [10]:
# Import model layers

from keras.models import Model
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, LeakyReLU
from keras import activations

In [11]:
# model_4

def create_model_4():
    inp_shape = (width, height, 3)
    filter_size = 64
    
    def conv_block(inp, i):
        conv = Conv2D(filter_size * (2**i), (3, 3), padding='same')(inp)
        conv = LeakyReLU()(conv)
        conv = Conv2D(filter_size * (2**i), (3, 3), padding='same')(conv)
        conv = LeakyReLU()(conv)
        pool = MaxPooling2D((2, 2))(conv)
        return pool, conv
    
    def bottleneck(inp, i):
        conv = Conv2D(filter_size * (2**i), (3, 3), padding='same')(inp)
        conv = LeakyReLU()(conv)
        conv = Conv2D(filter_size * (2**i), (3, 3), padding='same')(conv)
        conv = LeakyReLU()(conv)
        return conv
    
    def deconv_block(inp, shortcut, i):
        deconv = Conv2DTranspose(filter_size * (2**i), (3, 3), strides=(2, 2), padding="same")(inp)
        uconv  = concatenate([shortcut, deconv])
        uconv  = Conv2D(filter_size * (2**i), (3, 3), padding="same")(uconv)
        uconv  = LeakyReLU()(uconv)
        uconv  = Conv2D(filter_size * (2**i), (3, 3), padding="same")(uconv)
        uconv  = LeakyReLU()(uconv)
        return uconv
    
    #contracting half
    inp = Input(inp_shape, batch_size=BATCH_SIZE)
    conv0, pre_pool0 = conv_block(inp,   0)
    conv1, pre_pool1 = conv_block(conv0, 1)
    conv2, pre_pool2 = conv_block(conv1, 2)
    conv3, pre_pool3 = conv_block(conv2, 3)
    mid              = bottleneck(conv3, 4)
    deconv3          = deconv_block(mid,     pre_pool3, 3)
    deconv2          = deconv_block(deconv3, pre_pool2, 2)
    deconv1          = deconv_block(deconv2, pre_pool1, 1)
    deconv0          = deconv_block(deconv1, pre_pool0, 0)
    out = Conv2D(3, (1, 1), activation='sigmoid')(deconv0)
    model = Model(inputs=[inp], outputs=[out])
    return model

In [12]:
# Mixed gradient error
# https://medium.com/analytics-vidhya/loss-functions-for-image-super-resolution-sisr-8a65644fbd85

from keras.losses import MeanAbsoluteError, MeanSquaredError

mae = MeanAbsoluteError()
mse = MeanSquaredError()

def MeanGradientError(targets, outputs, weight):
    filter_x = tf.tile(tf.expand_dims(tf.constant([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype = np.float32), axis = -1), [1, 1, 3])
    filter_x = tf.tile(tf.expand_dims(filter_x, axis = -1), [1, 1, 1, 3])
    filter_y = tf.tile(tf.expand_dims(tf.constant([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype = np.float32), axis = -1), [1, 1, 3])
    filter_y = tf.tile(tf.expand_dims(filter_y, axis = -1), [1, 1, 1, 3])
    
    outputs = tf.transpose(outputs, perm=[0, 2, 1, 3])
    targets = tf.transpose(targets, perm=[0, 2, 1, 3])

    # output gradient
    output_gradient_x = tf.math.square(tf.nn.conv2d(outputs, filter_x, strides = 1, padding = 'SAME'))
    output_gradient_y = tf.math.square(tf.nn.conv2d(outputs, filter_y, strides = 1, padding = 'SAME'))

    #target gradient
    target_gradient_x = tf.math.square(tf.nn.conv2d(targets, filter_x, strides = 1, padding = 'SAME'))
    target_gradient_y = tf.math.square(tf.nn.conv2d(targets, filter_y, strides = 1, padding = 'SAME'))

    # square
    output_gradients = tf.math.sqrt(tf.math.add(output_gradient_x, output_gradient_y))
    target_gradients = tf.math.sqrt(tf.math.add(target_gradient_x, target_gradient_y))

    # compute mean gradient error
    mge = tf.math.reduce_sum(tf.math.squared_difference(output_gradients, target_gradients)) / (BATCH_SIZE * width * height)

    return mge * weight

def MixedGradientError(y_true, y_pred):
    return mse(y_true, y_pred) + MeanGradientError(y_true, y_pred, weight = 0.01)

In [13]:
# Compile model with Adam optimizer

model = create_model_4()
lr_schedule = keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=0.005, first_decay_steps=3*80, t_mul=2.0, m_mul=1.0)
opt = keras.optimizers.Adam(lr_schedule, epsilon=0.001)
model.compile(optimizer=opt, loss=MixedGradientError, metrics=["mean_squared_error", "mean_absolute_error"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(64, 256, 144, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (64, 256, 144, 64)   1792        ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (64, 256, 144, 64)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (64, 256, 144, 64)   36928       ['leaky_re_lu[0][0]']            
                                                                                              

In [14]:
# training model

import datetime

LOG_DIR = "/content/drive/MyDrive/model_256x144/logs/fit/" + "model4(" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ")"
#LOG_DIR = f"logs/fit/model2mae({datetime.datetime.now().strftime("%Y%m%d-%H%M%S")})"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=0)

MODEL_PATH = "/content/drive/MyDrive/model_256x144/"
if not os.path.exists(MODEL_PATH):
  os.makedirs(MODEL_PATH)
save_callback = tf.keras.callbacks.ModelCheckpoint(
    MODEL_PATH + "/weights.{epoch:02d}-{val_loss:.6f}_256-144.hdf5",
    save_weights_only=True
)

# model.load_weights("/content/drive/MyDrive/models/weights.50-0.185150_256-144.hdf5")
# model.load_weights("/content/drive/MyDrive/model_256x144/")

with tf.device("/GPU:0"):
    history = model.fit(gen_train,
                       epochs=93,
                       validation_data=gen_val,
                       callbacks=[tensorboard_callback, save_callback],
                       verbose=1
    )

Epoch 1/93
80/80 [==============================] - 694s 8s/step - loss: 0.0666 - mean_squared_error: 0.0238 - mean_absolute_error: 0.1157 - val_loss: 0.0342 - val_mean_squared_error: 0.0092 - val_mean_absolute_error: 0.0736
Epoch 2/93
80/80 [==============================] - 656s 8s/step - loss: 0.0297 - mean_squared_error: 0.0062 - mean_absolute_error: 0.0550 - val_loss: 0.0275 - val_mean_squared_error: 0.0052 - val_mean_absolute_error: 0.0486
Epoch 3/93
80/80 [==============================] - 654s 8s/step - loss: 0.0277 - mean_squared_error: 0.0051 - mean_absolute_error: 0.0476 - val_loss: 0.0271 - val_mean_squared_error: 0.0049 - val_mean_absolute_error: 0.0468
Epoch 4/93
80/80 [==============================] - 655s 8s/step - loss: 0.0264 - mean_squared_error: 0.0046 - mean_absolute_error: 0.0448 - val_loss: 0.0250 - val_mean_squared_error: 0.0044 - val_mean_absolute_error: 0.0429
Epoch 5/93
80/80 [==============================] - 659s 8s/step - loss: 0.0254 - mean_squared_error

KeyboardInterrupt: ignored

In [ ]:
print(history.history.keys())
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
while True:
  pass